In [19]:
from langgraph.graph import StateGraph,START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from typing import TypedDict,Annotated
from pydantic import BaseModel, Field
import operator

In [20]:
load_dotenv()

True

In [21]:
model = ChatGoogleGenerativeAI(model='gemini-2.5-pro')

In [22]:
class EvaluationSchema(BaseModel):

    feedback: str = Field(description='Detailed feedback for the essay')
    score: int = Field(description='Score out of 10', ge=0, le=10)

In [23]:
structured_model = model.with_structured_output(EvaluationSchema)

In [24]:
essay = """Artificial Intelligence (AI) has become an integral part of our daily lives, transforming the way we work, communicate, and interact with the world. From the smartphones in our pockets to the recommendation systems on streaming platforms, AI quietly powers many of the technologies we take for granted. Its ability to analyze vast amounts of data and recognize patterns has made it an essential tool in fields such as healthcare, finance, and transportation. By automating repetitive tasks, AI not only improves efficiency but also frees up human potential for more creative and strategic endeavors.

In the healthcare industry, AI is revolutionizing diagnostics, patient care, and drug discovery. Algorithms trained on medical data can now detect diseases like cancer or diabetic retinopathy earlier and more accurately than traditional methods. Virtual assistants and chatbots help manage patient queries, while robotic surgeries offer precision that reduces recovery time. AI also accelerates the development of new medications by simulating how different compounds interact with the human body, significantly shortening research timelines. These advancements are not just technological milestones but also steps toward more accessible and effective healthcare systems.

Beyond healthcare, AI plays a significant role in shaping modern economies and industries. In finance, AI systems detect fraudulent transactions, assess credit risk, and offer personalized financial advice. In manufacturing, predictive maintenance powered by AI helps prevent equipment failure and reduce downtime. AI's influence is also seen in agriculture, where smart systems monitor soil conditions, optimize irrigation, and predict crop yields. These applications not only enhance productivity but also support sustainability by minimizing waste and conserving resources. As industries embrace AI, they unlock new opportunities for growth and innovation.

However, the rise of AI also brings challenges that society must address thoughtfully. Concerns around data privacy, algorithmic bias, and job displacement are increasingly prominent. While AI has the potential to create new jobs and industries, it may also render certain roles obsolete, particularly those involving routine tasks. Ensuring that AI systems are transparent, fair, and aligned with human values is crucial for building trust and preventing harm. As we move forward, the focus should be on using AI responsibly—leveraging its power to improve lives while safeguarding ethical standards and human dignity.
"""

In [25]:
# structured_model.invoke(prompt)

In [26]:
class EssayState(TypedDict):

    essay: str
    language_feedback: str
    analysis_feedback: str
    clarity_feedback: str
    overall_feedback: str
    individual_scores: Annotated[list[int], operator.add]
    avg_score: float

In [27]:
def evaluate_language(state: EssayState):

    prompt = f'Evaluate the language quality of the following essay and provide a feedback and assign a score out of 10 \n {essay}'
    output = structured_model.invoke(prompt)

    return {'language_feedback': output.feedback, 'individual_scores': [output.score]}

In [28]:
def evaluate_analysis(state: EssayState):

    prompt = f'Evaluate the depth of analysis of the following essay and provide a feedback and assign a score out of 10 \n {essay}'
    output = structured_model.invoke(prompt)

    return {'analysis_feedback': output.feedback, 'individual_scores': [output.score]}

In [29]:
def evaluate_thought(state: EssayState):

    prompt = f'Evaluate the clarity of thought of the following essay and provide a feedback and assign a score out of 10 \n {essay}'
    output = structured_model.invoke(prompt)

    return {'clarity_feedback': output.feedback, 'individual_scores': [output.score]}

In [30]:
def final_evaluation(state: EssayState):

    # summary feedback
    prompt = f'Based on the following feedbacks create a summarized feedback \n language feedback - {state["language_feedback"]} \n depth of analysis feedback - {state["analysis_feedback"]} \n clarity of thought feedback - {state["clarity_feedback"]}'
    overall_feedback = model.invoke(prompt).content

    # avg calculate
    avg_score = sum(state['individual_scores'])/len(state['individual_scores'])

    return {'overall_feedback': overall_feedback, 'avg_score': avg_score}

In [32]:
graph = StateGraph(EssayState)

graph.add_node('evaluate_language', evaluate_language)
graph.add_node('evaluate_analysis', evaluate_analysis)
graph.add_node('evaluate_thought', evaluate_thought)
graph.add_node('final_evaluation', final_evaluation)

# edges
graph.add_edge(START, 'evaluate_language')
graph.add_edge(START, 'evaluate_analysis')
graph.add_edge(START, 'evaluate_thought')

graph.add_edge('evaluate_language', 'final_evaluation')
graph.add_edge('evaluate_analysis', 'final_evaluation')
graph.add_edge('evaluate_thought', 'final_evaluation')

graph.add_edge('final_evaluation', END)

workflow = graph.compile()

In [ ]:
initial_state = {
    'essay': """Artificial Intelligence (AI) has become an integral part of our daily lives, transforming the way we work, communicate, and interact with the world. From the smartphones in our pockets to the recommendation systems on streaming platforms, AI quietly powers many of the technologies we take for granted. Its ability to analyze vast amounts of data and recognize patterns has made it an essential tool in fields such as healthcare, finance, and transportation. By automating repetitive tasks, AI not only improves efficiency but also frees up human potential for more creative and strategic endeavors.

In the healthcare industry, AI is revolutionizing diagnostics, patient care, and drug discovery. Algorithms trained on medical data can now detect diseases like cancer or diabetic retinopathy earlier and more accurately than traditional methods. Virtual assistants and chatbots help manage patient queries, while robotic surgeries offer precision that reduces recovery time. AI also accelerates the development of new medications by simulating how different compounds interact with the human body, significantly shortening research timelines. These advancements are not just technological milestones but also steps toward more accessible and effective healthcare systems.

Beyond healthcare, AI plays a significant role in shaping modern economies and industries. In finance, AI systems detect fraudulent transactions, assess credit risk, and offer personalized financial advice. In manufacturing, predictive maintenance powered by AI helps prevent equipment failure and reduce downtime. AI's influence is also seen in agriculture, where smart systems monitor soil conditions, optimize irrigation, and predict crop yields. These applications not only enhance productivity but also support sustainability by minimizing waste and conserving resources. As industries embrace AI, they unlock new opportunities for growth and innovation."""
}

final_state = workflow.invoke(initial_state)

print(final_state)

{'essay': "Artificial Intelligence (AI) has become an integral part of our daily lives, transforming the way we work, communicate, and interact with the world. From the smartphones in our pockets to the recommendation systems on streaming platforms, AI quietly powers many of the technologies we take for granted. Its ability to analyze vast amounts of data and recognize patterns has made it an essential tool in fields such as healthcare, finance, and transportation. By automating repetitive tasks, AI not only improves efficiency but also frees up human potential for more creative and strategic endeavors.\n\nIn the healthcare industry, AI is revolutionizing diagnostics, patient care, and drug discovery. Algorithms trained on medical data can now detect diseases like cancer or diabetic retinopathy earlier and more accurately than traditional methods. Virtual assistants and chatbots help manage patient queries, while robotic surgeries offer precision that reduces recovery time. AI also acc